# This is an LSTM on the last word of a stream

In [1]:
from __future__ import print_function
import os
import pickle
import numpy as np
import pandas as pd
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/')

In [3]:
w2indx = pickle.load(open('./0-output/LSTMDQN_Dic_token2id.pkl', 'rb'))
indx2w = pickle.load(open('./0-output/LSTMDQN_Dic_id2token.pkl', 'rb'))

In [4]:
len(w2indx), len(indx2w)

(239692, 239692)

In [35]:
qdf = pd.read_csv('./0-output/nuggets_numtext.csv', nrows=56)

In [5]:
adf = pd.read_csv('./0-output/2012_aurora_shooting_numtext.csv', nrows=50000)

In [6]:
rans = np.random.random((adf.shape[0],1)) >0.9
rans2 = np.random.random((adf.shape[0],1)) >0.9
rans3 = (rans==False)*rans2
X_train, X_test = adf.values[rans], adf.values[rans3]

In [7]:
X_train = np.asarray([ [int(j) for j in i.split(" ")] for i in X_train])
X_test = np.asarray([ [int(j) for j in i.split(" ")] for i in X_test])

In [8]:
X_train.shape, X_test.shape

((4949, 2731), (4531, 2731))

In [11]:
emb_dim = 10
batch_size = 32

Y_train = np.random.choice([0,1], size=(len(X_train),1))
Y_test = np.random.choice([0,1], size=(len(X_test),1))

In [ ]:
# # Taking the last element of X_train[i] as our outcome
# Y_train = np.asarray([i[-1] for i in X_train]) 
# Y_test = np.asarray([ i[-1] for i in X_test])

# # Removing the last element of X_train[i]
# X_train = np.asarray([i[:-1] for i in X_train]) 
# X_test = np.asarray([i[:-1] for i in X_test] )

In [13]:
# print([x for x in X_train[0]])

In [15]:
print ([indx2w[x] for x in X_train[0] if x != 0])

[u'The', u'23rd', u'annual', u'Courage', u'Classic', u'cycling', u'tour', u'is', u'Saturday', u'Sunday', u'and', u'Monday', u'taking', u'cyclists', u'through', u'157', u'miles', u'across', u'Summit', u'County']


In [16]:
len(np.unique(X_train)), len(np.unique(Y_train))

(14179, 2)

In [18]:
# print(indx2w[Y_train[0]])

In [19]:
maxlen=40

In [20]:
# RNN for the regular embedings
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
# The pad sequences stuff seems suspicious
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

4949 train sequences
4531 test sequences
Pad sequences (samples x time)
X_train shape: (4949, 40)
X_test shape: (4531, 40)


In [30]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(indx2w), emb_dim)) # , dropout=0.2))
model.add(LSTM(100))
model.add(Dense(2))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

optimizer = RMSprop(lr=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=5,
          validation_data=(X_test, Y_test), verbose=0)

# Running the predictions
out = model.evaluate(X_test, Y_test, batch_size=batch_size)
ypred_tst = model.predict(X_test)

Build model...
Train...
4531/4531 [==============================] - 4s     
